In [1]:
import numpy as np
import pandas as pd


from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

from psql_functions import execQuery
from miss_data import add_missing_dates, add_missing_counts
from make_plots import dict_error_plot_epsilons, dict_error_plot_n
from sample_range_query import load_range_queries
import warnings
warnings.filterwarnings('ignore')
from collections.abc import Iterable

import functools 

import sys
import os
import re
owd = os.getcwd()

In [2]:
param_dic = {
    "host"      : "localhost",
    "database"  : "bachelorBesoeg2014",
    "user"      : "postgres",
    "password"  : "password",
    "port"      : "5432"
}

query = """select time_ from _775147;"""
result = execQuery(param_dic, query)
dates = [(date[0]) for date in result]

query = """select count_ from _775147;"""
result = execQuery(param_dic, query)

counts = [(count[0]) for count in result]

all_dates = add_missing_dates(dates)
all_counts =  add_missing_counts(counts, dates, all_dates)

n_data_structures = 50


Executed query and closed connection.
Executed query and closed connection.


In [3]:
onlydirs = [f for f in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(os.getcwd(), f)) and f[0] != '.']

In [4]:
onlydirs

['Data', 'plots', 'range_queries', 'results', '__pycache__']

In [5]:
def is_non_zero_file(fpath):  
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

def load_range_queries(path, file):
    if is_non_zero_file(path + file):
        return pd.read_csv(path + file, sep='\n',header=None).to_numpy().flatten()
    return []

In [6]:
def make_results_lookup_dict_n_r(epsilons, n, rs):
    epsilon_dict = dict.fromkeys(epsilons)
    n_dicts = []
    for i in range(0,len(epsilons)):
        n_dict = dict.fromkeys(n)
        n_dicts.append(n_dict)
    r_dicts = []
    for i in range(0,len(n)*len(epsilons)):
        r_dict = dict.fromkeys(rs[i%len(n)])
        r_dicts.append(r_dict)
    i = 0
    j = 0
    for e_key in epsilon_dict:
        epsilon_dict[e_key] = n_dicts[i]
        n_dict = epsilon_dict[e_key]
        for n_key in n_dict:
            #print(j)
            n_dict[n_key] = r_dicts[j]
            j = j + 1
        i = i + 1
    return epsilon_dict


def fill_up_results_lookup_dict_n_r(epsilons, n, rs, path, files):
    dict_ = make_results_lookup_dict_n_r(epsilons, n, rs)
    for idx, f in enumerate(files):
        e = re.search(r'e=[-+]?[0-9]*\.?[0-9]*', f).group(0)
        e = e.split("e=")[1]
        N = re.search(r'N=\d+', f).group(0)
        N = N.split("N=")[1]
        r = re.search(r'r=\d+', f).group(0) 
        r = r.split("r=")[1]

        dict_[float(e)][int(N)][int(r)] = load_range_queries(path, f)
    return dict_


In [7]:
def make_query_lookup_dict_n_r(n, rs):
    n_dict = dict.fromkeys(n)
    r_dicts = []
    for i in range(0,len(n)):
        #print(rs[i])
        r_dict = dict.fromkeys(rs[i])
        r_dicts.append(r_dict)

    for idx, n_key in enumerate(n_dict):
            n_dict[n_key] = r_dicts[idx]
    
    return n_dict

def fill_up_query_lookup_dict_n_r(n, rs, path, files):
    dict_ = make_query_lookup_dict_n_r(n, rs)
    for idx, f in enumerate(files):
        N = re.search(r'N=\d+', f).group(0)
        N = N.split("N=")[1]
        r = re.search(r'r=\d+', f).group(0) 
        r = r.split("r=")[1]
        
        querries = load_range_queries(path, f)
        dict_[int(N)][int(r)] = querries.copy()
    return dict_

In [8]:
def make_results_lookup_dict_n_degree(epsilons, n, degrees):
    epsilon_dict = dict.fromkeys(epsilons)
    n_dicts = []
    for i in range(0,len(epsilons)):
        n_dict = dict.fromkeys(n)
        n_dicts.append(n_dict)
    degree_dicts = []
    for i in range(0,len(n)*len(epsilons)):
        degree_dict = dict.fromkeys(degrees)

        degree_dicts.append(degree_dict)
    i = 0
    j = 0
    for e_key in epsilon_dict:
        #print(e_key)
        epsilon_dict[e_key] = n_dicts[i]
        n_dict = epsilon_dict[e_key]
        #print(epsilon_dict[e_key])
        for n_key in n_dict:
            n_dict[n_key] = degree_dicts[j]
            j = j + 1
        i = i + 1
        #print(epsilon_dict)
    return epsilon_dict

def fill_up_results_lookup_dict_n_degree(epsilons, n, degrees, path, files):
    dict_ = make_results_lookup_dict_n_degree(epsilons, n, degrees)
    for idx, f in enumerate(files):
        e = re.search(r'e=[-+]?[0-9]*\.?[0-9]*', f).group(0)
        e = e.split("e=")[1]
        N = re.search(r'N=\d+', f).group(0)
        N = N.split("N=")[1]
        B = re.search(r'B=\d+', f).group(0) 
        B = B.split("B=")[1]

        dict_[float(e)][int(N)][int(B)] = load_range_queries(path, f)
    return dict_

In [9]:
def make_query_lookup_dict_n_degree(n, degrees):
    n_dict = dict.fromkeys(n)
    degree_dicts = []
    for i in range(0,len(n)):
        degree_dict = dict.fromkeys(degrees)
        degree_dicts.append(degree_dict)

    for idx, n_key in enumerate(n_dict):
            n_dict[n_key] = degree_dicts[idx]
    
    return n_dict

def fill_up_query_lookup_dict_n_degree(n, degrees, path, files):
    dict_ = make_query_lookup_dict_n_degree(n, degrees)
    for idx, f in enumerate(files):
        here_n = re.search(r'N=\d+', f).group(0)
        here_n = here_n.split("N=")[1]
        here_degree = re.search(r'B=\d+', f).group(0) 
        here_degree = here_degree.split("B=")[1]
        
        querries = load_range_queries(path, f)
        dict_[int(here_n)][int(here_degree)] = querries.copy()
    return dict_

In [10]:
def make_results_lookup_dict_n(epsilons, n):
    epsilon_dict = dict.fromkeys(epsilons)
    n_dicts = []
    for i in range(0,len(epsilons)):
        n_dict = dict.fromkeys(n)
        n_dicts.append(n_dict)
    degree_dicts = []
    for i in range(0,len(n)*len(epsilons)):
        degree_dict = dict.fromkeys(degrees)

        degree_dicts.append(degree_dict)
    i = 0
    j = 0
    for e_key in epsilon_dict:
        #print(e_key)
        epsilon_dict[e_key] = n_dicts[i]
        n_dict = epsilon_dict[e_key]
        #print(epsilon_dict[e_key])
        for n_key in n_dict:
            n_dict[n_key] = degree_dicts[j]
            j = j + 1
        i = i + 1
        #print(epsilon_dict)
    return epsilon_dict

def make_results_lookup_dict_n(epsilons, n, path, files):
    dict_ = make_results_lookup_dict_n_degree(epsilons, n, degrees)
    for idx, f in enumerate(files):
        e = re.search(r'e=[-+]?[0-9]*\.?[0-9]*', f).group(0)
        e = e.split("e=")[1]
        N = re.search(r'N=\d+', f).group(0)
        N = N.split("N=")[1]

        dict_[float(e)][int(N)] = load_range_queries(path, f)
    return dict_


def fill_up_query_lookup_dict_n(n, path, files):
    dict_ = dict.fromkeys(n)
    for idx, f in enumerate(files):
        here_n = re.search(r'N=\d+', f).group(0)
        here_n = here_n.split("N=")[1]
        
        querries = load_range_queries(path, f)
        dict_[int(here_n)] = querries.copy()
    return dict_

In [11]:
def load_split_est_n_cor_files(path):
    files_cor = [f for f in os.listdir(path) if re.match(r'(cor).*\.csv', f)]
    files_est = [f for f in os.listdir(path) if re.match(r'(est).*\.csv', f)]

    #Where flat beats hh
    cor_flat_better = [f for f in files_cor if re.match(r'.*(_flat_).*\.csv', f)]
    #Where hh beats flat
    cor_hh_better = flat_files_cor = [f for f in files_cor if re.match(r'.*(_hh_).*\.csv', f)]

    #Where flat beats hh
    est_flat_better =  [f for f in files_est if re.match(r'.*(_flat_).*\.csv', f)]
    #Where hh beats flat
    est_hh_better = [f for f in files_est if re.match(r'.*(_hh_).*\.csv', f)]
    
    return est_flat_better, cor_flat_better, est_hh_better, cor_hh_better

In [12]:
def check_queries_var(epsilons, n, qurey_dict, est_dict, cor_dict):
    for e in epsilons:
        for N in n:
            for var in qurey_dict[N]:

                if qurey_dict[N][var] is not None:

                    if (len(qurey_dict[N][var]) == len(cor_dict[e][N][var]) == len(est_dict[e][N][var])):
                        if not functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,qurey_dict[N][var],cor_dict[e][N][var]), True): 
                            print ("Correct dict; the lists are not the same") 
                        if not functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,qurey_dict[N][var],est_dict[e][N][var]), True): 
                            print ("Est dict; The lists are not the same")
                    else:
                        print('Not the same length')

    return 'All queries match'

def check_queries_flat(epsilons, n, qurey_dict, est_dict, cor_dict):
    for e in epsilons:
        for N in n:
            if qurey_dict[N] is not None:

                if (len(qurey_dict[N]) == len(cor_dict[e][N]) == len(est_dict[e][N])):
                    if not functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,qurey_dict[N],cor_dict[e][N]), True): 
                        print ("Correct dict; the lists are not the same") 
                    if not functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,qurey_dict[N],est_dict[e][N]), True): 
                        print ("Est dict; The lists are not the same")
                else:
                    print('Not the same length')

    return 'All queries match'

In [13]:
epsilons = np.array([2, 1.4, 1.2, 1, 0.8, 0.6, 0.5, 0.3])
n = np.array([32,128,256,512,1024,2048])
degrees = np.array([2,3,4])

flat_r_32 =  np.array([2, 4, 8, 12, 16, 20, 24])
flat_r_128 =  np.array([20, 40, 50, 60, 70, 80, 90])
flat_r_256 =  np.array([40, 60, 80, 100, 140, 200, 220])
flat_r_512 =  np.array([100, 150, 200, 250, 300, 400, 450])
flat_r_1024 =  np.array([200, 300, 400, 500, 600, 800, 900])
flat_r_2048 =  np.array([600, 800, 1000, 1250, 1500, 1700, 1800])

rs = [flat_r_32, flat_r_128, flat_r_256, flat_r_512, flat_r_1024, flat_r_2048]

# Check if correct queries for varying size of r

## Loading the data 

In [14]:
#C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src\results\sample_querys\local_hh
path = owd +'/'+'results/sample_querys/flat_varying_r/'

In [15]:
print(path)

C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src/results/sample_querys/flat_varying_r/


In [16]:
files = os.listdir(path)
files_cor = files[:len(files)//2]
files_est = files[len(files)//2:]
print(files)

['cor_e=0.3_N=1024_r=200.csv', 'cor_e=0.3_N=1024_r=300.csv', 'cor_e=0.3_N=1024_r=400.csv', 'cor_e=0.3_N=1024_r=500.csv', 'cor_e=0.3_N=1024_r=600.csv', 'cor_e=0.3_N=1024_r=800.csv', 'cor_e=0.3_N=1024_r=900.csv', 'cor_e=0.3_N=128_r=20.csv', 'cor_e=0.3_N=128_r=40.csv', 'cor_e=0.3_N=128_r=50.csv', 'cor_e=0.3_N=128_r=60.csv', 'cor_e=0.3_N=128_r=70.csv', 'cor_e=0.3_N=128_r=80.csv', 'cor_e=0.3_N=128_r=90.csv', 'cor_e=0.3_N=2048_r=1000.csv', 'cor_e=0.3_N=2048_r=1250.csv', 'cor_e=0.3_N=2048_r=1500.csv', 'cor_e=0.3_N=2048_r=1700.csv', 'cor_e=0.3_N=2048_r=1800.csv', 'cor_e=0.3_N=2048_r=600.csv', 'cor_e=0.3_N=2048_r=800.csv', 'cor_e=0.3_N=2048_r=900.csv', 'cor_e=0.3_N=256_r=100.csv', 'cor_e=0.3_N=256_r=140.csv', 'cor_e=0.3_N=256_r=200.csv', 'cor_e=0.3_N=256_r=220.csv', 'cor_e=0.3_N=256_r=40.csv', 'cor_e=0.3_N=256_r=60.csv', 'cor_e=0.3_N=256_r=80.csv', 'cor_e=0.3_N=32_r=12.csv', 'cor_e=0.3_N=32_r=16.csv', 'cor_e=0.3_N=32_r=2.csv', 'cor_e=0.3_N=32_r=20.csv', 'cor_e=0.3_N=32_r=24.csv', 'cor_e=0.3_N=3

In [17]:
cor_dict = fill_up_results_lookup_dict_n_r(epsilons, n, rs, path, files_cor)
est_dict = fill_up_results_lookup_dict_n_r(epsilons, n, rs, path, files_est)


In [18]:
cor_dict

{2.0: {32: {2: array(['2014-01-21, 2014-01-22', '2014-01-26, 2014-01-27',
          '2014-01-29, 2014-01-30', ..., '2014-01-28, 2014-01-29',
          '2014-01-05, 2014-01-06', '2014-01-14, 2014-01-15'], dtype=object),
   4: array(['2014-01-19, 2014-01-22', '2014-01-24, 2014-01-27',
          '2014-01-09, 2014-01-12', ..., '2014-01-13, 2014-01-16',
          '2014-01-08, 2014-01-11', '2014-01-12, 2014-01-15'], dtype=object),
   8: array(['2014-01-05, 2014-01-12', '2014-01-19, 2014-01-26',
          '2014-01-08, 2014-01-15', ..., '2014-01-10, 2014-01-17',
          '2014-01-20, 2014-01-27', '2014-01-23, 2014-01-30'], dtype=object),
   12: array(['2014-01-15, 2014-01-26', '2014-01-04, 2014-01-15',
          '2014-01-07, 2014-01-18', ..., '2014-01-07, 2014-01-18',
          '2014-01-08, 2014-01-19', '2014-01-19, 2014-01-30'], dtype=object),
   16: array(['2014-01-08, 2014-01-23', '2014-01-11, 2014-01-26',
          '2014-01-02, 2014-01-17', ..., '2014-01-09, 2014-01-24',
          '2014-0

In [19]:
path =  owd +'/'+'range_queries/flat_varying_r/'
files = os.listdir(path)
"""
print(files)

files = [f for f in path if re.match(r'.*\.csv', f)]
print(files)
"""
flat_n_r_dict = fill_up_query_lookup_dict_n_r(n, rs, path, files)

In [20]:
print(check_queries_var(epsilons, n, flat_n_r_dict, est_dict, cor_dict))


All queries match


# Checking for flat model

## hh queries

In [21]:
path = owd +'/'+'range_queries/local_hh/'
query_files = os.listdir(path)

#query_files = [f for f in path if re.match(r'.*\.csv', f)]
local_hh_queries = fill_up_query_lookup_dict_n_degree(n, degrees, path, query_files)

## Flat queries

In [22]:
path = owd +'/'+'range_queries/flat/'
query_files = os.listdir(path)
flat_queries = fill_up_query_lookup_dict_n(n, path, query_files)

In [23]:
flat_queries

{32: array(['2014-01-07, 2014-01-12', '2014-01-04, 2014-01-09',
        '2014-01-14, 2014-01-19', ..., '2014-01-27, 2014-02-01',
        '2014-01-19, 2014-01-24', '2014-01-24, 2014-01-29'], dtype=object),
 128: array(['2014-01-14, 2014-01-22', '2014-01-29, 2014-02-06',
        '2014-01-17, 2014-01-25', ..., '2014-04-23, 2014-05-01',
        '2014-02-11, 2014-02-19', '2014-03-29, 2014-04-06'], dtype=object),
 256: array(['2014-07-10, 2014-08-02', '2014-06-01, 2014-06-24',
        '2014-03-29, 2014-04-21', ..., '2014-05-20, 2014-06-12',
        '2014-08-02, 2014-08-25', '2014-05-05, 2014-05-28'], dtype=object),
 512: array(['2015-04-03, 2015-05-07', '2014-04-18, 2014-05-22',
        '2014-01-19, 2014-02-22', ..., '2014-08-17, 2014-09-20',
        '2014-02-01, 2014-03-07', '2014-04-24, 2014-05-28'], dtype=object),
 1024: array(['2015-06-03, 2015-07-16', '2016-01-11, 2016-02-23',
        '2015-04-09, 2015-05-22', ..., '2015-05-31, 2015-07-13',
        '2014-07-12, 2014-08-24', '2014-03-15,

In [24]:
#Split files in correct chunks
path = owd +'/'+'results/sample_querys/flat/'
flat_files_est_flat_better, flat_files_cor_flat_better, flat_files_est_hh_better, flat_files_cor_hh_better = load_split_est_n_cor_files(path)


In [25]:
path = owd +'/'+'results/sample_querys/flat/'
flat_files_hh_cor_dict = fill_up_results_lookup_dict_n_degree(epsilons, n, degrees, path, flat_files_cor_hh_better)
flat_files_hh_est_dict = fill_up_results_lookup_dict_n_degree(epsilons, n, degrees, path, flat_files_est_hh_better)
flat_files_flat_cor_dict = make_results_lookup_dict_n(epsilons, n, path, flat_files_cor_flat_better)
flat_files_flat_est_dict = make_results_lookup_dict_n(epsilons, n, path, flat_files_est_flat_better)
print(check_queries_var(epsilons, n, local_hh_queries, flat_files_hh_cor_dict, flat_files_hh_est_dict))
print(check_queries_flat(epsilons, n, flat_queries, flat_files_flat_cor_dict, flat_files_flat_est_dict))

All queries match
All queries match


# Checking for hh model

In [26]:
path = owd +'/'+'results/sample_querys/local_hh/'
hh_files_est_flat_better, hh_files_cor_flat_better, hh_files_est_hh_better, hh_files_cor_hh_better = load_split_est_n_cor_files(path)

In [30]:
flat_files_hh_cor_dict = fill_up_results_lookup_dict_n_degree(epsilons, n, degrees, path, hh_files_cor_hh_better)
flat_files_hh_est_dict = fill_up_results_lookup_dict_n_degree(epsilons, n, degrees, path, hh_files_est_hh_better)
flat_files_flat_cor_dict = make_results_lookup_dict_n(epsilons, n, path, hh_files_cor_flat_better)
flat_files_flat_est_dict = make_results_lookup_dict_n(epsilons, n, path, hh_files_est_flat_better)
print(check_queries_var(epsilons, n, local_hh_queries, flat_files_hh_cor_dict, flat_files_hh_est_dict))
print(check_queries_flat(epsilons, n, flat_queries, flat_files_flat_cor_dict, flat_files_flat_est_dict))

All queries match
All queries match
